In [ ]:
"""
Purpose: To Create the table that
will store the neuron objects that have finer
axon preprocessing

"""

In [24]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

In [25]:
#so that it will have the adapter defined
from datajoint_utils import *

In [26]:
test_mode = True

# Debugging the contains method

In [27]:
import system_utils as su

In [28]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

Sleeping 126 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


In [29]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

# Defining the Table

In [30]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 6997


# Proofreading Version

In [31]:
@schema
class DecompositonAxonVersion(dj.Manual):
    definition="""
    axon_version      : tinyint unsigned  # key by which to lookup the finer axon processing method
    ---
    description          : varchar(256)    # new parts of the finer axon preprocessing
    """
versions=[[0,"axon with standard meshparty"],
          [2,"axon with finer resolution"],
         [4,"even more fine resoution, axon skeleton, boutons, webbing"],
         [5,"filtered away floating pieces near soma for stitching"],
         [6, "max stitch distance = 2000, face threshold = 50"]]

dict_to_write = [dict(axon_version=k,description=v) for k,v in versions]
DecompositonAxonVersion.insert(dict_to_write,skip_duplicates=True)

DecompositonAxonVersion()

axon_version key by which to lookup the finer axon processing method,description new parts of the finer axon preprocessing
0,axon with standard meshparty
2,axon with finer resolution
4,"even more fine resoution, axon skeleton, boutons, webbing"
5,filtered away floating pieces near soma for stitching
6,"max stitch distance = 2000, face threshold = 50"


In [33]:
minnie.DecompositionCellType()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,allen_e_i,allen_e_i_n_nuc,allen_cell_type,allen_cell_type_n_nuc,allen_cell_type_e_i,dendrite_on_axon_merges_error_area the area (in um ^ 2) of the faces canceled out by filter,dendrite_on_axon_merges_error_length the length (in um) of skeleton distance canceled out by filter,cell_type_for_axon,axon_volume volume of the oriented bounding box of axon (divided by 10^14),axon_length length (in um) of the classified axon skeleton,axon_branch_length_median length (in um) of the classified axon skeleton,axon_branch_length_mean length (in um) of the classified axon skeleton,axon_n_branches,axon_n_short_branches,axon_n_long_branches,axon_n_medium_branches,axon_bbox_x_min,axon_bbox_y_min,axon_bbox_z_min,axon_bbox_x_max,axon_bbox_y_max,axon_bbox_z_max,axon_bbox_x_min_soma_relative,axon_bbox_y_min_soma_relative,axon_bbox_z_min_soma_relative,axon_bbox_x_max_soma_relative,axon_bbox_y_max_soma_relative,axon_bbox_z_max_soma_relative,run_time the amount of time to run (seconds)
864691134988385914,0,0.25,30.00,5,0,0,6,=BLOB=,610607,12260.15,1,1,excitatory,densely_spined,1,1,166.32176177458544,0.0004958699646598768,0,1112802.6425228184,60,1888801.5329428245,excitatory,1,None,None,None,0.0,0.0,excitatory,1017.1908875511735,2077.7584374888374,8.80683277733624,29.68226339269768,70,26,6,38,1193651.4138548293,429508.5806022349,522108.7649461699,1440289.6219283065,1060078.232233446,1176155.8877907617,-193392.46151859942,-73947.33616682998,-325572.0027670375,53245.74655487784,556622.3154643811,328475.12007755425,337.72
864691134988385914,0,0.25,30.00,5,0,1,6,=BLOB=,610607,1401.24,1,1,excitatory,densely_spined,1,0,171.0319294846302,0.0007133259607713956,0,418676.15770513995,40,940619.5929469143,excitatory,1,None,None,None,0.0,0.0,excitatory,187.8039963250148,1607.1076036830723,6.281401625586479,15.75595689885365,102,49,2,51,1283838.7017211474,469608.0888050826,773279.0359224229,1443341.0082371572,1037049.4954368536,980778.4326581849,-98608.95131782768,-25317.325933265616,-68684.98314271902,60893.355198182166,542124.0806985054,138814.413593043,241.18
864691134988386682,0,0.25,30.00,7,0,0,6,=BLOB=,269380,1067.32,1,1,excitatory,densely_spined,1,0,136.20429783298385,0.0003973503610348998,0,2356426.512289788,46,2840861.4322549435,excitatory,1,5P_IT,1,excitatory,0.0,0.0,excitatory,218.24937591197695,1976.87370

In [34]:
# minnie,schema = du.configure_minnie_vm()
# minnie.DecompositionAxon.delete()
minnie.DecompositionCellType.delete()
minnie.schema.external['decomposition'].delete(delete_external_files=True)

About to delete:
`microns_minnie65_02`.`__decomposition_cell_type`: 157 items
Proceed? [yes, No]: yes
Committed.


100%|██████████| 157/157 [00:01<00:00, 94.35it/s] 


[]

In [17]:
import numpy as np
import time
import classification_utils as clu
import proofreading_utils as pru
import axon_utils as au
import synapse_utils as syu
import preprocessing_vp2 as pre
import cell_type_utils as ctu

axon_version = au.axon_version
ver = 88

verbose = True
validation = True
inh_exc_class_to_use_for_axon = "Baylor"

@schema
class DecompositionCellType(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    -> minnie.DecompositonAxonVersion()             # the version of code used for this cell typing classification
    ---

    # -- attributes for the cell type classification ---
    decomposition        : <decomposition> # saved neuron object with high fidelity axon


    # ----- Nucleus Information ----#
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.
    nuclei_distance      : double                    # the distance to the closest nuclei (even if no matching nuclei found)
    n_nuclei_in_radius   : tinyint unsigned          # the number of nuclei within the search radius of 15000 belonging to that segment
    n_nuclei_in_bbox     : tinyint unsigned          # the number of nuclei within the bounding box of that soma


    # ------ Information Used For Excitatory Inhibitory Classification (Baylor Cell) -------- 
    cell_type_predicted: enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier
    spine_category: enum('no_spined','sparsely_spined','densely_spined')

    n_axons: tinyint unsigned             # Number of axon candidates identified
    n_apicals: tinyint unsigned             # Number of apicals identified

    axon_angle_maximum=NULL:double #the anlge of an identified axon
    spine_density_classifier:double              # the number of spines divided by skeletal length for branches analyzed in classification
    n_branches_processed: int unsigned                 # the number branches used for the spine density analysis
    skeletal_length_processed: double                 # The total skeletal length of the viable branches used for the spine density analysis
    n_branches_in_search_radius: int unsigned                 # the number branches existing in the search radius used for spine density
    skeletal_length_in_search_radius : double         # The total skeletal length of the branches existing in the search radius used for spine density


    #---- allen classification info -----
    allen_e_i=NULL: enum('excitatory','inhibitory','other','unknown')
    allen_e_i_n_nuc=NULL: tinyint unsigned  
    allen_cell_type=NULL:varchar(256)
    allen_cell_type_n_nuc=NULL:tinyint unsigned  
    allen_cell_type_e_i=NULL:enum('excitatory','inhibitory','other','unknown')


    # ----- for the dendrite on axon filtering away --------
    dendrite_on_axon_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    dendrite_on_axon_merges_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter

    # ----- attributes from the axon_features --- 
    cell_type_for_axon : enum('excitatory','inhibitory','other','unknown')
    axon_volume=NULL: double #volume of the oriented bounding box of axon (divided by 10^14)

    axon_length=NULL: double  # length (in um) of the classified axon skeleton
    axon_branch_length_median=NULL: double  # length (in um) of the classified axon skeleton
    axon_branch_length_mean=NULL: double  # length (in um) of the classified axon skeleton

    # number of branches in the axon
    axon_n_branches=NULL: int unsigned  
    axon_n_short_branches=NULL:  int unsigned
    axon_n_long_branches=NULL:  int unsigned
    axon_n_medium_branches=NULL:  int unsigned

    #bounding box features
    axon_bbox_x_min=NULL: double 
    axon_bbox_y_min=NULL: double 
    axon_bbox_z_min=NULL: double 
    axon_bbox_x_max=NULL: double 
    axon_bbox_y_max=NULL: double 
    axon_bbox_z_max=NULL: double 

    axon_bbox_x_min_soma_relative=NULL: double 
    axon_bbox_y_min_soma_relative=NULL: double 
    axon_bbox_z_min_soma_relative=NULL: double 
    axon_bbox_x_max_soma_relative=NULL: double 
    axon_bbox_y_max_soma_relative=NULL: double 
    axon_bbox_z_max_soma_relative=NULL: double 

    run_time=NULL : double                   # the amount of time to run (seconds)
    """
                             
    
    #key_source = minnie.Decomposition() & minnie.NucleiSegmentsRun2() & "segment_id=864691136540183458"
    #key_source = minnie.Decomposition() & du.proofreading_segment_id_restriction() & "segment_id=864691134884753146"
    key_source = (minnie.Decomposition() & 
                  du.current_validation_segment_id_restriction
                  - du.current_validation_segment_id_exclude)
    

    def make(self,key):
        """
        Pseudocode:
        1) Pull Down all the Neuron Objects associated with a segment_id
        
        For each neuron:
        2) Run the full axon preprocessing
        3) Save off the neuron
        4) Save dict entry to list
        
        
        5) Write the new entry to the table

        """
        
        
        # 1) Pull Down All of the Neurons
        segment_id = key["segment_id"]
        
        if verbose:
            print(f"------- Working on Neuron {segment_id} -----")
        
        whole_pass_time = time.time()
        
        #1) Pull Down all the Neuron Objects associated with a segment_id
        neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id,
                                                                            ignore_DecompositionAxon=True,
                                                                            ignore_DecompositionCellType = True)

        if verbose:
            print(f"Number of Neurons found ={len(neuron_objs)}")

        #For each neuron:
        dict_to_write = []
        
        
        # -------- getting the nuclei info to match
#         ver = 88
#         nucleus_ids,nucleus_centers = du.segment_to_nuclei(segment_id,
#                                                                nuclei_version=ver)
        segment_map_dict = (minnie.AutoProofreadValidationSegmentMap4() & dict(old_segment_id=segment_id)).fetch1()
        nucleus_id = segment_map_dict["nucleus_id"]
        nuc_center_coords = du.nuclei_id_to_nucleus_centers(nucleus_id)
        
        nucleus_ids = [nucleus_id]
        nucleus_centers = [nuc_center_coords]
        
        print(f"nucleus_ids = {nucleus_ids}")
        print(f"nucleus_centers = {nucleus_centers}")
        
        for split_index,neuron_obj in zip(neuron_split_idxs,neuron_objs):
            
            if verbose:
                print(f"--> Working on Split Index {split_index} -----")
                
            st = time.time()
            
            
            # ------------- Does all of the processing -------------------
            
            #1) ------ Getting the paired nuclei ------
            winning_nucleus_id, nucleus_info = nru.pair_neuron_obj_to_nuclei(neuron_obj,
                                     "S0",
                                      nucleus_ids,
                                      nucleus_centers,
                                     nuclei_distance_threshold = 15000,
                                      return_matching_info = True,
                                     verbose=True)
            
            # else:
            #     winning_nucleus_id = 12345
            #     nucleus_info = dict()
            #     nucleus_info["nucleus_id"] = winning_nucleus_id
            #     nucleus_info["nuclei_distance"] = 0
            #     nucleus_info["n_nuclei_in_radius"] = 1
            #     nucleus_info["n_nuclei_in_bbox"] = 1

            if verbose:
                print(f"nucleus_info = {nucleus_info}")
                print(f"winning_nucleus_id = {winning_nucleus_id}")
        

            #2) ------- Finding the Allen Cell Types -------
            allen_cell_type_info = ctu.allen_nuclei_classification_info_from_nucleus_id(winning_nucleus_id)
            if verbose:
                print(f"allen_cell_type_info = {allen_cell_type_info}")

            
            #4) -------- Running the cell classification and stats--------------
            
            if verbose:
                print(f"\n\n ------ Part C: Inhibitory Excitatory Classification ---- \n\n")

            filter_time = time.time()

            (inh_exc_class,
             spine_category,
             axon_angles,
             n_axons,
             n_apicals,
             neuron_spine_density,
             n_branches_processed,
             skeletal_length_processed,
             n_branches_in_search_radius,
             skeletal_length_in_search_radius
             ) = clu.inhibitory_excitatory_classifier(neuron_obj,
                                                return_spine_classification=True,
                                                return_axon_angles=True,
                                                 return_n_axons=True,
                                                 return_n_apicals=True,
                                                 return_spine_statistics=True,
                                                     axon_limb_branch_dict_precomputed=None,
                                                axon_angles_precomputed=None,
                                                     verbose=verbose)
            if verbose:
                print(f"Total time for classification = {time.time() - filter_time}")

            all_axon_angles = []
            for limb_idx,limb_data in axon_angles.items():
                for candidate_idx,cand_angle in limb_data.items():
                    all_axon_angles.append(cand_angle)

            if len(axon_angles)>0:
                axon_angle_maximum = np.max(all_axon_angles)
            else:
                axon_angle_maximum = 0


            if verbose:
                print("\n -- Cell Type Classification Results --")
                print(f"inh_exc_class={inh_exc_class}")
                print(f"spine_category={spine_category}")
                print(f"axon_angles={axon_angles}")
                print(f"n_axons={n_axons}")
                print(f"n_apicals={n_apicals}")
                print(f"neuron_spine_density={neuron_spine_density}")
                print(f"n_branches_processed={n_branches_processed}")
                print(f"skeletal_length_processed={skeletal_length_processed}")
                print(f"n_branches_in_search_radius={n_branches_in_search_radius}")
                print(f"skeletal_length_in_search_radius={skeletal_length_in_search_radius}")

            baylor_cell_type_info = dict(
                        cell_type_predicted=inh_exc_class,
                         spine_category=spine_category,
                        axon_angle_maximum = axon_angle_maximum,
                         n_axons=n_axons,
                         n_apicals=n_apicals,
                         spine_density_classifier=neuron_spine_density,
                         n_branches_processed=neuron_spine_density,
                         skeletal_length_processed=skeletal_length_processed,
                         n_branches_in_search_radius=n_branches_in_search_radius,
                         skeletal_length_in_search_radius=skeletal_length_in_search_radius,

            )

            
            
            #5) ----- Deciding on cell type to use for axon 
            e_i_class = inh_exc_class
            if inh_exc_class_to_use_for_axon == "Allen" and allen_cell_type_info["e_i"] is not None:
                e_i_class = allen_cell_type_info["e_i"]

            if verbose:
                print(f"e_i_class = {e_i_class} with inh_exc_class_to_use_for_axon = {inh_exc_class_to_use_for_axon}")

            
            
            #6) -------- If excitatory running the axon processing--------------
            """
            Psuedocode: 
            If e_i class is excitatory:
            1) Filter away the axon on dendrite
            2) Do the higher fidelity axon processing
            3) Compute the axon features

            """
            
            if e_i_class == "excitatory" and neuron_obj.axon_limb_name is not None:
                if verbose:
                    print(f"Excitatory so performing high fidelity axon and computing axon features")
            #     1) Filter away the axon on dendrite
            #     2) Do the higher fidelity axon processing

                o_neuron, filtering_info = au.complete_axon_processing(neuron_obj,
                                                                      perform_axon_classification = False,
                                                                      return_filtering_info = True)
                filtering_info = {k:np.round(v,2) for k,v in filtering_info.items() if "area" in k or "length" in k}
                #3) Compute the axon features
                axon_features = au.axon_features_from_neuron_obj(o_neuron)
            else:
                nru.clear_all_branch_labels(neuron_obj,labels_to_clear="axon")
                o_neuron = neuron_obj
                axon_features = dict()
                filtering_info = dict()


            
            #3) ------ Adding the Synapses -----------
            o_neuron = syu.add_synapses_to_neuron_obj(o_neuron,
                            validation = validation,
                            verbose  = True,
                            original_mesh = None,
                            plot_valid_error_synapses = False,
                            calculate_synapse_soma_distance = False,
                            add_valid_synapses = True,
                              add_error_synapses=False)
            
            
            
            # ------- Saving off the neuron object ----------------
            save_time = time.time()
            ret_file_path = o_neuron.save_compressed_neuron(
                                            output_folder=str(du.get_decomposition_path()),
                                            #output_folder = "./",
            file_name=f"{o_neuron.segment_id}_{split_index}_split_axon_v{au.axon_version}",
                                              return_file_path=True,
                                             export_mesh=False,
                                             suppress_output=True)

            ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
            
            if verbose:
                print(f"ret_file_path_str = {ret_file_path_str}")
                print(f"Save time = {time.time() - save_time}")
            
            n_dict = dict(key,
              split_index = split_index,
              axon_version = au.axon_version,
             decomposition=ret_file_path_str,
              run_time = np.round(time.time() - st,2),
            cell_type_for_axon = e_i_class,
             )
            
            dicts_for_update = [baylor_cell_type_info,
                                allen_cell_type_info,
                                nucleus_info,
                                filtering_info,
                                axon_features]
            
            for d in dicts_for_update:
                n_dict.update(d)

            self.insert1(n_dict,skip_duplicates=True,allow_direct_insert=True)
            #dict_to_write.append(n_dict)
        
        #write the
        #self.insert(dict_to_write,skip_duplicates=True,allow_direct_insert=True)

        print(f"\n\n ***------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------ ***")

# Running the Populate

In [22]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_cell_type'")
curr_table#.delete()
#((curr_table & "status='error'") & "key_hash='c4c2c7d5612554a90a9ece804e84d1e1'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_cell_type,3a9977fe0309921114d7ad948205d19d,error,=BLOB=,IndexError: arrays used as indices must be of integer (or boolean) type,=BLOB=,celiib@10.28.0.133,at-node6,1,549056,2021-06-23 13:01:00
__decomposition_cell_type,48a5aef7c4abd7fced95d3dbeed140ed,error,=BLOB=,IndexError: arrays used as indices must be of integer (or boolean) type,=BLOB=,celiib@10.28.0.129,at-node2,1,548994,2021-06-23 13:07:30
__decomposition_cell_type,88cc23cf175d74be27a56afd0262035c,error,=BLOB=,"Exception: Segment ID 864691136672696199 should have been in DecompositionSplit but wasnt with n_error_limbs = 4, n_somas = 2",=BLOB=,celiib@10.28.0.175,at-node48,1,548991,2021-06-23 13:05:46


In [23]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)
du = reload(du)
ctu = reload(ctu)
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionCellType.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    DecompositionCellType.populate(reserve_jobs=True, suppress_errors=False,)# order="random")
print('Populate Done')

print(f"Total time for DecompositionCellType populate = {time.time() - start_time}")


Populate Started
------- Working on Neuron 864691135501833053 -----
**Using table __decomposition_split for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Number of Neurons found = 1
Number of Neurons found =1
nucleus_ids = [608166]
nucleus_centers = [array([1382976,  402624,  753080])]
--> Working on Split Index 0 -----
soma_center = [1380711  402601  752672]
nuclei_within_radius = [608166]
nuclei_within_radius_distance = [2301.56859555]

There were 1 nuclei found within the radius of 15000 nm
winning_nuclei = 608166
winning_nuclei_distance = 2301.568595545221

 For Bounding Box Search:
inside_nuclei = [608166]


At End: using return_id_0_if_no_matches = True
winning_nuclei = 608166
winning_nuclei_distance = 2301.568595545221
n_nuclei_in_radius = 1
n_nuclei_in_bbox = 1
nucleus_info = {'nucleus_id': 608166, 'nuclei_distance': 2301.57, 'n_nuclei_in_radius': 1, 'n_nuclei_in_bbox': 1}
winning_nucleus_id = 608166
allen_cell_type_info = {'allen_e_i': 'excit

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:16<00:00, 16.40s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 17.95294213294983
connecting at the root
branches_touching_root = [495]
combining close nodes
combine_threshold = 1300
length of Graph = 40768
After combining close endpoints max(kept_branches_idx) = 495, len(kept_branches_idx) = 425
empty_indices % = 0.0
 conflict_indices % = 0.029117136820504513



AFTER face_lookup_resolved_test


...




--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (1, 2)
empty_indices % = 0.0973630831643002
 conflict_indices % = 0.0015212981744421906



AFTER face_lookup_resolved_test



main_branch = 224
The closest float distance was 205393.51156132386 which was greater than the maximum stitch distance 2000
 --> so ending the floating mesh stitch processs
Limb Correspondence AFTER stitching = 490
Recieved another instance of Neuron class in init -- so just copying data
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7f0c6b38a1d0>]}
Assigning the old width calculation because no valid new widths


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1372842.   407909.5  749183. ]---------
Starting_edge inside branches_to_conept = [[1370716.   408793.   748777.2]
 [1372842.   407909.5  749183. ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [411]
printing out current edge:
[[1372842.   407909.5  749183. ]
 [1370716.   408793.   748777.2]]
edge_endpoints_to_process was empty so exiting loop after 490 iterations
starting_node in concept map (that s


Assigning the old width calculation because no valid new widths




---Step 1: Computing synapse_dict---
# of beginning_direct_connections = 6960 
# of presyn: 232
# of postsyn: 6728

---Step 2: Computing mesh_label_dict---
-- Working on presyn--
Using original_mesh_method
# of distance_errored_syn_idx = 0
# of mesh_errored_syn_idx = 0
# of valid_syn_idx = 232
-- Working on postsyn--
Using original_mesh_method
# of distance_errored_syn_idx = 0
# of mesh_errored_syn_idx = 11
# of valid_syn_idx = 6717

---Step 3: add_valid_synapses_to_neuron_obj---
Synapse dict: 0.0029
Original Mesh: 0.9677
Closest Branch: 15.9411
Closest Skeleton Branch and distance from endpoint: 9.4618
Closest Branch: 23.7328
Closest Skeleton Branch and distance from endpoint: 163.9076
Total time for valid synapse objects = 0.0507047176361084
Saving Neuorn in suppress_output mode...please wait
Saved File at location: /mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135501833053_0_split_axon_v6
ret_file_path_str = /mnt/dj-stor01/platinum/minnie65/02/decomposition/8646911355018